In [1]:
import os
import json
import argparse
import yaml
import warnings

from utils import load_model, generate_answer, Retrieve_documents
from retriever import Index_Builder

from transformers import AutoTokenizer
import pandas as pd

from secret import api_key
import guidance
warnings.filterwarnings(action='ignore')

from prompts import refine_query_prompt, confidence_prompt, default_prompt, strategy_ext_prompt, answer_prompt


In [2]:
def main(args, verbose_document: bool = False, verbose_strategy: bool = False):
    
    refined_query = query = args['query']   
    
    # structure_program = guidance(refine_query_prompt, query = query)
    # res_refine = generate_answer(args['model_name'], api_key, refine_query_prompt, query = query)
    
    # refined_query = res_refine['refined_query']

    sparse_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    
    raw_reference_list = Retrieve_documents(
                            query = query, 
                            data_dir = args['data_dir'],
                            sparse_tokenizer = sparse_tokenizer,
                            use_content_type = args['content_type'],
                            top_k = args['top_k'], 
                            check_point_dir = args['check_point_dir'],
                            index_save_dir = args['index_save_dir'],
                            retrieve_mode = args['retrieve_mode'] #'sparse'
                            )
                     
    if verbose_document:       
        print(raw_reference_list)                   
           
    # res_confidence = generate_answer(args['model_name'], api_key, confidence_prompt, query = refined_query, 
    #                                 passage = [raw_reference_list[0]]) #has to be a list
    
    # confidence = res_confidence['confidence'].lower()
    
    confidence = 'yes'
    
    if confidence == 'no':
        res_default = generate_answer(args['model_name'], api_key, default_prompt, query = refined_query)
        strategy = 'default'
        answer = res_default['answer']
        
    else:
        
        try:
            res_strategy = generate_answer(args['model_name'], api_key, strategy_ext_prompt, query = refined_query, 
                                        passages = [raw_reference_list[0]]) # has to be a list
            
            strategy = res_strategy['strategy']
            
        except: # Split Passage into two when given passage is too long
            
            try: 
                first_half = raw_reference_list[0]['contents'][:int(len(raw_reference_list[0])*0.5)]
                last_half = raw_reference_list[0]['contents'][int(len(raw_reference_list[0])*0.5):]
            
            except:
                first_half = raw_reference_list[0]['content'][:int(len(raw_reference_list[0])*0.5)]
                last_half = raw_reference_list[0]['content'][int(len(raw_reference_list[0])*0.5):]
            
            
            res_strategy = generate_answer(args['model_name'], api_key, strategy_ext_prompt, query = refined_query, 
                                passages = [first_half]) # has to be a list
            
            strategy = res_strategy['strategy']
            
            # res_strategy_2 = generate_answer(args['model_name'], api_key, strategy_ext_prompt, query = refined_query, 
            #         passages = [last_half]) # has to be a list
            
            # strategy_2 = res_strategy_2['strategy']
            
            # strategy = strategy + '\n' + strategy_2
            
        if verbose_strategy:
            print(strategy)
        
        res_answer = generate_answer(args['model_name'], api_key, answer_prompt, query = refined_query,
                                     strategy = strategy) # has to be a list

        first_answer = res_answer['answer']
        #final_answer = res_answer['final_answer']
                                
        answer = first_answer
        
    save = {'Question': query, 
            'Confidence': confidence,
            'Strategy': strategy,
            'Reference': raw_reference_list,
            'Answer': answer}  
    
    print(save)  
        
    return answer

In [3]:
args = yaml.load(open("/Project/src/config/runner_chatgpt.yaml",'r'), Loader=yaml.FullLoader)

In [5]:
args['query'] = '요즘 맨유가 아무리 돈을 써도 안되는 이유'

main(args, verbose_document = True, verbose_strategy = True)

100%|██████████| 388296/388296 [00:00<00:00, 915780.56it/s]


./index_save_dir/sparse/bm25_index.pickle
Loading Saved sparse index
Searching Sparse Index...
[{'title': '홀란드 몸값 2300억 원…스콜스 "맨유가 데려올 수 있다"', 'url': 'https://sports.news.naver.com/news?oid=477&aid=0000293829', 'contents': '\n▲ 엘링 홀란드.[스포티비뉴스=맹봉주 기자] 아무리 비싸도 영입하려는 팀들은 있다. 이번 여름 이적 시장의 주인공은 엘링 홀란드(21)다. 이적 시장 소문의 대부분이 홀란드와 관련되어 있다. 보루시아 도르트문트는 이미 홀란드를 내놓았다. 책정한 몸값은 1억 5000만 파운드(약 2300억 원). 축구 이적료로는 네이마르가 바르셀로나에서 PSG(파리생제르맹)로 갈 때 기록한 2960억 원 다음으로 높은 금액이다.홀란드의 아버지와 에이전트 미노 라이올라는 진작에 협상 테이블을 차렸다. 바르셀로나, 레알 마드리드, 맨체스터 유나이티드, 맨체스터 시티, 바이에른 뮌헨, 첼시, 유벤투스 등 거론되는 팀들은 한둘이 아니다.맨유의 전설, 폴 스콜스는 이중 홀란드가 갈만한 팀으로 딱 두 곳을 집는다. 8일(한국 시간) 영국 방송 \'BT 스포츠\'에 나온 스콜스는 "맨유가 홀란드를 영입할 수 있을까?"란 질문을 받자 "충분히 가능하다고 본다. 그 돈을 내서 홀란드를 데려올 수 있다면, 그렇게 하지 않을 이유가 없다"며 고개를 끄덕였다.다른 팀들이 홀란드를 데려가기 힘든 이유도 설명했다. "중요한 건 홀란드 몸값을 감당할 여력이 있는 팀이 어디냐는 것이다. 레알 마드리드와 바르셀로나는 저 정도의 돈을 지불할 능력이 안 된다는 소리가 들린다"며 "맨시티는 돈은 있지만, 천문학적인 지출을 하고 싶어 하지 않는다. 무엇보다 홀란드는 펩 과르디올라 맨시티 감독이 선호하는 유형의 선수가 아니다. 과르디올라 감독은 홀란드같은 공격수를 잘 기용하지 않는다. 올 시즌 맨시티는 세르히오 아게로나

'맨체스터 유나이티드(맨유)가 많은 돈을 쓰더라도 성과를 내지 못하는 이유는 여러 가지입니다. \n\n첫째, 맨유는 충분한 자금을 가지고 있지만, 선수를 영입하는데 있어서 경제적인 여력만이 중요한 것은 아닙니다. 감독의 선수 선호도도 중요한 요소입니다. 예를 들어, 펩 과르디올라 맨시티 감독은 홀란드같은 공격수를 잘 기용하지 않는다는 사례가 있습니다. 이처럼 감독의 선수 선호도가 맞지 않으면, 많은 돈을 들여 선수를 영입하더라도 성과를 내기 어렵습니다.\n\n둘째, 선수를 영입하는데 있어서 경쟁 팀들과의 경쟁도 중요한 요소입니다. 맨유가 원하는 선수를 영입하려면 다른 팀들과의 경쟁에서 이겨야 합니다. 그러나 이는 쉽지 않은 일입니다. 예를 들어, 홀란드를 영입하려는 팀 중에는 첼시도 있습니다. 이처럼 경쟁 팀들과의 경쟁에서 밀리면, 많은 돈을 써도 원하는 선수를 영입하기 어렵습니다.\n\n이러한 이유들로 인해 맨유는 많은 돈을 쓰더라도 원하는 성과를 내기 어렵습니다.'